In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
from data_agent import *
import numpy as np
from data_util import *
import tensorflow as tf
from sklearn_model_hyper import *
import pandas as pd
from stock_agent import *
from backtest import *
from joblib import dump, load
from datetime import datetime, timedelta
from tec_an import *
from bitstamp import *
from process_live_trade import *
from model_winner_select import *

base_dir = "data/"

In [2]:
currency = "btcusd"

full_data = base_dir + "backtest/"
full_data = base_dir + "4jul21/"
#full_data = base_dir + "btcusd/"
data_gen = DataGenerator(random = False, base_dir = full_data)
data_gen.rewind()
data_count = (data_gen.steps - 100)
#data_count = 200000
raw = data_gen.next()

minute = 30

In [3]:
path = "model/"
files = os.listdir(path)
models = []
for file in files:
    models.append(f"{path}{file}")

models = ['model/rand']
    
timestamp = int(raw['timestamp'])

In [4]:
winner = get_best_model(
    minutes_list=[minute],
    result_paths=models,
    currency_list=[currency],
    timestamp = timestamp,
    winner_path = None
)

  0%|          | 0/10 [00:00<?, ?it/s]

model/rand - 10
Pre selected: 10
https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=1800&limit=1000&start=1625258981
Downloaded from 2021-07-02 21:30:00 to 2021-07-04 23:30:00 35655.00
Resample 30Min - TecAn ( windows 40, windows_limit 100 )
Processing 100 of btcusd


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:15<00:00,  1.57s/it]

Selected: 9
Pipeline(steps=[('normalizer', Normalizer()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=ExtraTreesClassifier(max_features=0.3),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(bootstrap=False,
                                                         criterion='entropy',
                                                         max_features=0.1,
                                                         random_state=42))]))])
btcusd -> 103.36863496765659 | 21 - 9
{'recall': 0.5177065767284992, 'precision': 0.5265866209262435, 'f1': 0.5221088435374149, 'accuracy': 0.6102635228848821, 'roc_auc': 0.5963091187529421}

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
               

In [5]:
result = winner
model = result['model']
window = result['window']
minutes = result['minutes']
step = result['step']
profit = result['profit']
print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
print(f"{model}")

agent, back, stock = get_agent(minutes = minutes,
                                win = window,
                                step = step,
                                currency = currency,
                                hot_load = False,
                                model = model,
                                timestamp = timestamp,
                                simulate_on_price = True)

Minutes=30 Window=40 Step=2 | 140.6828226509521
Pipeline(steps=[('normalizer', Normalizer()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=ExtraTreesClassifier(max_features=0.3),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(bootstrap=False,
                                                         criterion='entropy',
                                                         max_features=0.1,
                                                         random_state=42))]))])
Resample 30Min - TecAn ( windows 40, windows_limit 100 )


In [6]:
import numpy as np
import matplotlib.pyplot as plt
def get_charts(stock_data):
    fig = plt.figure()
    ax = plt.axes()
    x = stock_data
    plt.plot(x, linewidth=1)
    #plt.grid(b=True, color='aqua', alpha=0.3, linestyle='-.', linewidth=2)
    plt.show()


In [7]:
val_start = 0
val_end = 0
def get_online_data(minutes, 
                    source_data_generator, 
                    load_from_disk, file_prefix = "",
                    val_start = val_start,
                    val_end = val_end):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = [currency],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = val_start
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    online.load_val_cache([currency], online.val_start, online.val_end)
    
    return online

In [8]:
prices = []
agent.on_closed_price = lambda price: prices.append(price)

on_raw_data = lambda raw: print(raw)


In [9]:
data_gen.rewind()


first = data_gen.next()
#data_count
for i in range(data_count):
    raw = data_gen.next()
    agent.on_new_raw_data(raw)

2021-08-13 23:29:06.804785 ModelAgent(35340.23): 2021-07-04 23:00:04(1625439604) UP
2021-08-13 23:29:06.804983 BackTest: 2021-07-04 23:00:04 Buy (35340.23): ask: 35340.23
2021-08-13 23:29:13.864161 ModelAgent(35415.34): 2021-07-04 23:30:08(1625441408) UP
2021-08-13 23:29:13.864351 BackTest: 2021-07-04 23:30:08 Profit detected price:35415.34  bid:35415.34 ask:35415.34
2021-08-13 23:29:13.864410 BackTest: 2021-07-04 23:30:08 SOLD > (35415.34) Result: PROFIT 0.2125 total: 100.21253398746978
2021-08-13 23:29:20.325248 ModelAgent(35247.7): 2021-07-05 00:00:17(1625443217) UP
2021-08-13 23:29:20.325422 BackTest: 2021-07-05 00:00:17 Buy (35247.7): ask: 35247.7
2021-08-13 23:29:27.583643 ModelAgent(34922.82): 2021-07-05 00:30:08(1625445008) UP
2021-08-13 23:29:34.229493 ModelAgent(34898.82): 2021-07-05 01:00:40(1625446840) UP
2021-08-13 23:29:40.664884 ModelAgent(34639.91): 2021-07-05 01:30:15(1625448615) DOWN
2021-08-13 23:29:40.665104 BackTest: 2021-07-05 01:30:15 SOLD > (34639.91) Result: LO

In [10]:
back.sell(back.buy_price)
back.report()

2021-08-13 23:31:16.726217 BackTest: 2021-07-05 08:50:02 SOLD > (34216.83) Result: LOSS 0.0 total: 97.83045928919479
-2.16954% -> 97.83045928919479
States: 301794 Positive: 1 Negative: 4


In [11]:
#-2.16954% -> 97.83045928919479
#States: 301794 Positive: 1 Negative: 4

In [24]:
#agent.ohlc.plot()

In [13]:
val_end = int(raw['timestamp'])
val_start = int(first['timestamp'])

#val_start = 1625439604
#val_end = 1625473811

In [25]:
#get_charts(prices)
print(len(prices))

20


In [15]:
tec = TecAn(windows = window, windows_limit = 100)
source_data_generator = SourceDataGenerator(tec = tec)

In [16]:
online = get_online_data(minutes = minute, 
                 source_data_generator = source_data_generator, 
                 load_from_disk = False, 
                 file_prefix = "____",
                 val_start = val_start,
                 val_end = val_end)

https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=1800&limit=1000&start=1625438981
Downloaded from 2021-07-04 23:30:00 to 2021-07-05 10:00:00 34256.42
Resample 30Min - TecAn ( windows 40, windows_limit 100 )
Processing 21 of btcusd


  0%|          | 0/21 [00:00<?, ?it/s]

In [17]:
x, y = online.load_val_data(currency)

In [26]:
#get_charts(y)
print(len(y))

21


In [19]:
prices

[35340.23,
 35415.34,
 35247.7,
 34922.82,
 34898.82,
 34639.91,
 34560.8,
 34477.16,
 34430.21,
 34426.56,
 34363.72,
 34245.31,
 34199.5,
 34112.89,
 34344.29,
 34299.57,
 34295.01,
 34154.96,
 34269.56,
 34174.91]

In [20]:
y

array([35340.23, 35415.34, 35255.  , 34922.82, 34898.82, 34639.91,
       34560.8 , 34477.16, 34430.21, 34426.56, 34363.72, 34245.31,
       34199.5 , 34112.89, 34344.29, 34299.57, 34295.01, 34149.71,
       34269.56, 34174.91, 34256.42])

In [21]:
back = test_model(model, currency, online, step, verbose = True)

2021-08-13 23:31:18.327497 BackTest: 1970-01-01 00:00:00 Buy (35340.23): ask: 35340.23
2021-08-13 23:31:18.340488 BackTest: 1970-01-01 00:00:00 Profit detected price:35415.34  bid:35415.34 ask:35415.34
2021-08-13 23:31:18.340601 BackTest: 1970-01-01 00:00:00 SOLD > (35415.34) Result: PROFIT 0.2125 total: 100.21253398746978
2021-08-13 23:31:18.364099 BackTest: 1970-01-01 00:00:00 Buy (34922.82): ask: 34922.82
2021-08-13 23:31:18.388536 BackTest: 1970-01-01 00:00:00 SOLD > (34639.91) Result: LOSS -0.8101 total: 99.40243331404194
2021-08-13 23:31:18.400114 BackTest: 1970-01-01 00:00:00 Buy (34560.8): ask: 34560.8
2021-08-13 23:31:18.433955 BackTest: 1970-01-01 00:00:00 SOLD > (34426.56) Result: LOSS -0.3884 total: 99.01401637924876
2021-08-13 23:31:18.479052 BackTest: 1970-01-01 00:00:00 Buy (34112.89): ask: 34112.89
2021-08-13 23:31:18.494946 BackTest: 1970-01-01 00:00:00 Profit detected price:34344.29  bid:34344.29 ask:34344.29
2021-08-13 23:31:18.495094 BackTest: 1970-01-01 00:00:00 SO

In [22]:
back.report()

-0.42017% -> 99.57982868095408
States: 21 Positive: 2 Negative: 3
